In [16]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching comf_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s comf_Scene"

In [2]:
# Parameter
m1 = 5
m2 = 5

r1 = np.sqrt(m1/PI/10)
r2 = np.sqrt(m2/PI/10)

b = 0.4
pos1 = np.array([-5., 0.])
pos2 = np.array([0., pos1[1] - b])

v1_general = np.array([10., 0.])
v2_general = np.array([0., 0.])

print(r1, r2)

0.3989422804014327 0.3989422804014327


In [3]:
# schiefer Stoß
def push_pos_i(T, dt):
    N = int(T/dt)

    b = 0.4
    pos1 = np.array([-5., 0.])
    pos2 = np.array([0., pos1[1] - b])

    v1 = v1_general
    v2 = v2_general

    pos1_array = np.zeros((N+1, 3))
    pos2_array = np.zeros((N+1, 3))

    pos1_array[0,:2] = pos1
    pos2_array[0,:2] = pos2

    i = 1

    # Konstante Geschwindigkeit von m1 bis Treffen auf m2
    while ((r1 + r2) < npl.norm(pos1 - pos2)):
        pos1 += v1 * dt
        pos2 += v2 * dt

        pos1_array[i,:2] = pos1
        pos2_array[i,:2] = pos2

        i += 1

    # Vollkommen elastischer Stoß von m1 auf m2
    theta2 = np.arccos(np.dot(pos2-pos1, v1)/(npl.norm(pos2-pos1)*npl.norm(v1)))
    theta = PI - 2*theta2
    v2 = (pos2 - pos1) / npl.norm(pos2 - pos1) * npl.norm(v1) * 2*m1 / (m1 + m2) * np.sin(theta/2)  
    v1 = np.array([-v2[1], v2[0]]) / npl.norm(v2)  * npl.norm(v1) * np.sqrt(m1**2 + m2**2 + 2*m1*m2*np.cos(theta)) / (m1 + m2)

    t_break = i - 2
    print(v1, v2)

    # Konstante Geschwindigkeit m1 und m2 bis Ende des Arrays
    while (i <= N):
        pos1 += v1 * dt
        pos2 += v2 * dt

        pos1_array[i,:2] = pos1
        pos2_array[i,:2] = pos2

        i += 1

    return pos1_array, pos2_array, t_break

In [4]:
T = 10
dt = 0.01

#push_pos_i(T, dt)

In [24]:
%%capture_video --path "animations/center_of_mass_force/scattering_zoom.mp4"
%%manim -qm --fps 60 $video_scene

class comf_Scene(MovingCameraScene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        # Parameter
        T = 10
        dt = 0.01

        pos1_array, pos2_array, t_break = push_pos_i(T, dt)

        # Achsen zu Masse 2
        cat_x_axis = Line([-50, pos2_array[0][1], 0], [50, pos2_array[0][1], 0], color = WHITE, stroke_width = 0.5)
        cat_y_axis = Line([pos2_array[0][0], -50, 0], [pos2_array[0][0], 50, 0], color = WHITE, stroke_width = 0.5)

        cat_x = Text("x'", font_size = 16).move_to([6.75, pos2_array[0][1] + 0.25, 0])
        cat_y = Text("y'", font_size = 16).move_to([pos2_array[0][0] + 0.25, 3.75, 0])

        # 2 Massen
        m1_dot = Dot(radius = r1, color = BLUE).move_to(pos1_array[0])
        m2_dot = Dot(radius = r2, color = WHITE).move_to(pos2_array[0])

        # Laborpositionen als Iterator
        m1_dot.iter = iter(pos1_array)
        m2_dot.iter = iter(pos2_array)

        m1_dot.color = BLUE
        m2_dot.color = WHITE

        cat_x_axis.iter = iter(pos2_array)
        cat_y_axis.iter = iter(pos2_array)
        cat_x.iter = iter(pos2_array)
        cat_y.iter = iter(pos2_array)


        def mass_updater(mass):
            next_pos = next(mass.iter)
            con_line = Line(mass.get_center(), next_pos, color = mass.color, stroke_opacity = 0.5)
            self.add(con_line)
            mass.move_to(next_pos)

        def cat_x_axis_updater(axis):
            y = next(axis.iter)[1]
            axis.move_to([0, y, 0])

        def cat_y_axis_updater(axis):
            x = next(axis.iter)[0]
            axis.move_to([x, 0, 0])

        def cat_x_updater(label):
            y = next(label.iter)[1] + 0.25
            label.move_to([6.75, y, 0])

        def cat_y_updater(label):
            x = next(label.iter)[0] + 0.25
            label.move_to([x, 3.75, 0])


        self.add(m1_dot, m2_dot)
        self.add(cat_x_axis, cat_y_axis, cat_x, cat_y)

        m1_dot.add_updater(mass_updater)
        m2_dot.add_updater(mass_updater)

        cat_x_axis.add_updater(cat_x_axis_updater)
        cat_y_axis.add_updater(cat_y_axis_updater)
        cat_x.add_updater(cat_x_updater)
        cat_y.add_updater(cat_y_updater)

        # Save the state of camera
        self.camera.frame.save_state()

        time = ValueTracker(0)
        self.play(time.animate.set_value(0), rate_func = linear, run_time = t_break / 60)
        m1_dot.remove_updater(mass_updater)
        m2_dot.remove_updater(mass_updater)

        cat_x_axis.remove_updater(cat_x_axis_updater)
        cat_y_axis.remove_updater(cat_y_axis_updater)
        cat_x.remove_updater(cat_x_updater)
        cat_y.remove_updater(cat_y_updater)
        self.wait(0.5)
        # self.play(
        #     AnimationGroup(

        #     )
        #     self.camera.frame.set_width,m1_dot.get_width()*1.2,
        #     self.camera.frame.move_to,m1_dot
        #     )
        self.play(self.camera.frame.animate.set_height(m1_dot.height * 1.5))
        self.wait(2)

        self.play(time.animate.set_value(0), rate_func = linear, run_time = 3)

Output saved by overwring previous file at animations/center_of_mass_force/scattering_zoom.mp4.
[3.07692308 4.61538462] [ 6.92307692 -4.61538462]
